## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import random
import os

import lightgbm as lgb

from skimage import util, exposure
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
!pip install py7zr
import py7zr
import os

if not os.path.exists('/kaggle/train/') :
    os.makedirs('/kaggle/train/')


with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/train")


for dirname, _, filenames in os.walk('/kaggle'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

## Load dataset

In [ ]:
df_train = pd.read_json("/kaggle/train/data/processed/train.json")
df_train.head()

In [ ]:
def prepare_data(df):
    X_band_1 = []
    X_band_2 = []
    
    for band in df["band_1"]:
        #Convert to float32
        band_1 = np.array(band).astype(np.float32)
        #Reshaping band_1 and band_2
        band_1 = band_1.reshape(75,75)
        X_band_1.append(band_1)
        
    for band in df["band_2"]:
         #Convert to float32
        band_2 = np.array(band).astype(np.float32)
        #Reshaping band_1 and band_2
        band_2 = band_2.reshape(75,75)
        X_band_2.append(band_2)
        
    #Convert list to numpy array
    X_band_1 = np.array(X_band_1)
    X_band_2 = np.array(X_band_2)
    
    # Rescale
    X_band_1 = (X_band_1 - X_band_1.mean()) / (X_band_1.max() - X_band_1.min())
    X_band_2 = (X_band_2 - X_band_2.mean()) / (X_band_2.max() - X_band_2.min())
    
    #Concatenate band_1 and band_2 to create X for training (or test)
    X = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                        X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], 
                        axis=-1)
    
    Y = np.array(df["is_iceberg"])
    
    return X, Y

In [ ]:
X, Y = prepare_data(df_train)

print("X shape is:{}".format(X.shape))
print("Y shape is:{}".format(Y.shape))

# GLCM features and LGBM Image Classification

In [ ]:
# We need to convert the color to grey image for feature extraction
rgb_weights = [0.2989, 0.5870, 0.1140]
images = np.dot(X[...,:3], rgb_weights)
print("grayscale_image shape is:{}".format(images.shape))

# Visulalize gray image
plt.imshow(images[1], cmap=plt.get_cmap("gray"))
plt.show()

In [ ]:
# Note: before training the model using GLCM, You need to rescale the intensity to the range [0, 1] 
# before converting the image to 8-bit unsigned integer:
# because our data is float and we need to rescale that and then convert to integer
# Here, we want to do this process:
bin_width = 32

X_scaled = exposure.rescale_intensity(images, out_range=(0, 1))
X_binned = util.img_as_ubyte(X_scaled)
X_binned = X_binned // bin_width

In [ ]:
X_scaled

In [ ]:
X_binned

In [ ]:
SIZE = 75

# Split data to train and test
X_train, X_test, Y_train, Y_test = train_test_split(X_binned, Y, test_size=0.2, random_state=42)

print("X_train shape is:{}".format(X_train.shape))
print("Y_train shape is:{}".format(Y_train.shape))
print("X_test shape is:{}".format(X_test.shape))
print("Y_test shape is:{}".format(Y_test.shape))

In [ ]:
# Feature extraction
# In this case, we want to extract feature using gray-level co-occurrence matrix(GLCM)

def feature_extraction(data):
    img_ds = pd.DataFrame()
    
    # Iterate through images in file
    for img in range(data.shape[0]):
        # Create a data frame temperory to give features
        df = pd.DataFrame()
        image = data[img,:,:]
        
        # Now, we add data to data frame using GLCM 
        # GLCM = greycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        GLCM = greycomatrix(image, [1], [0])
        GLCM_energy = greycoprops(GLCM, "energy")[0]
        df["Energy"] = GLCM_energy
        GLCM_corr = greycoprops(GLCM, "correlation")[0]
        df["Correlation"] = GLCM_corr
        GLCM_diss = greycoprops(GLCM, "dissimilarity")[0]
        df["Dissimilarity"] = GLCM_diss
        GLCM_homgen = greycoprops(GLCM, "homogeneity")[0]
        df["Homogenity"] = GLCM_homgen
        GLCM_cont = greycoprops(GLCM, "contrast")[0]
        df["Contrast"] = GLCM_cont
        
        # GLCM 2
        GLCM2 = greycomatrix(image, [3], [0])
        GLCM_energy2 = greycoprops(GLCM2, "energy")[0]
        df["Energy2"] = GLCM_energy2
        GLCM_corr2 = greycoprops(GLCM2, "correlation")[0]
        df["Correlation2"] = GLCM_corr2
        GLCM_diss2 = greycoprops(GLCM2, "dissimilarity")[0]
        df["Dissimilarity2"] = GLCM_diss2
        GLCM_homgen2 = greycoprops(GLCM2, "homogeneity")[0]
        df["Homogeneity2"] = GLCM_homgen2
        GLCM_cont2 = greycoprops(GLCM2, "contrast")[0]
        df["Contrast2"] = GLCM_cont2
        
        # GLCM 3
        GLCM3 = greycomatrix(image, [5], [0])
        GLCM_energy = greycoprops(GLCM3, "energy")[0]
        df["Energy3"] = GLCM_energy
        GLCM_corr3 = greycoprops(GLCM3, "correlation")[0]
        df["Correlation3"] = GLCM_corr3
        GLCM_diss3 = greycoprops(GLCM3, "dissimilarity")[0]
        df["Dissimilarity3"] = GLCM_diss3
        GLCM_homgen3 = greycoprops(GLCM3, "homogeneity")[0]
        df["Homogeneity3"] = GLCM_homgen3
        GLCM_cont3 = greycoprops(GLCM3, "contrast")[0]
        df["Contrast3"] = GLCM_cont3
        
        # GLCM 4
        GLCM4 = greycomatrix(image, [0], [np.pi/4])
        GLCM_energy4 = greycoprops(GLCM4, "energy")[0]
        df["Energy4"] = GLCM_energy4
        GLCM_corr4 = greycoprops(GLCM4, "correlation")[0]
        df["Correlation4"] = GLCM_corr4
        GLCM_diss4 = greycoprops(GLCM4, "dissimilarity")[0]
        df["Dissimilarity4"] = GLCM_diss4
        GLCM_homgen4 = greycoprops(GLCM4, "homogeneity")[0]
        df["Homogeneity4"] = GLCM_homgen4
        GLCM_cont4 = greycoprops(GLCM4, "contrast")[0]
        df["Contrast4"] = GLCM_cont4
        
        # GLCM 5
        GLCM5 = greycomatrix(image, [0], [np.pi/4])
        GLCM_energy5 = greycoprops(GLCM5, "energy")[0]
        df["Energy5"] = GLCM_energy5
        GLCM_corr5 = greycoprops(GLCM5, "correlation")[0]
        df["Correlation5"] = GLCM_corr5
        GLCM_diss5 = greycoprops(GLCM5, "dissimilarity")[0]
        df["Dissimilarity5"] = GLCM_diss5
        GLCM_homgen5 = greycoprops(GLCM5, "homogeneity")[0]
        df["Homogeneity5"] = GLCM_homgen5
        GLCM_cont5 = greycoprops(GLCM5, "contrast")[0]
        df["Contrast5"] = GLCM_cont5
        
        # Append all features into img_ds
        img_ds = img_ds.append(df)
        
    return img_ds

In [ ]:
# Now, extract features from X_train
img_features = feature_extraction(X_train)
X_for_training = img_features

# Now, extract features from X_test
img_test_features = feature_extraction(X_test)
img_test_features = np.expand_dims(img_test_features, axis=0)
X_for_test = np.reshape(img_test_features, (X_test.shape[0], -1))

In [ ]:
print("Image Features shape is = {}".format(img_features.shape))

In [ ]:
X_for_training

# LGBM model

In [ ]:
# Note: in LGBM model, the classes number will start 0, for that reason, here we reassign labels 
# from 1,2,3,4 to 0,1,2,3 ect
dtrain = lgb.Dataset(X_for_training, label = Y_train)

# Set the lgbm parameters
# Note: here we are a binary classification problem, so 'objective' should be 'binary'
# while if we have multiple class, 'objective' would be 'multiclass'
lgbm_parameters = {"learning_rate":1.0,
                   "task":"train",
                   "boosting_type":"gbdt",
                   "objective":"binary",
                   "num_leaves":63,
                   "min_data_in_leaf":1,
                   "feature_fraction":1.0,
                   "bagging_freq":50,  
                   "bagging_fraction":0.8,
                   "max_depth":5,
                   "num_class":1}

In [ ]:
# Build the model
lgbm_model = lgb.train(lgbm_parameters, dtrain, 100)

In [ ]:
# Predict the model
prediction = lgbm_model.predict(X_for_test)
#prediction = np.argmax(prediction, axis=1)

In [ ]:
# Check the accuracy
# Note: prediction valuse is a probability (float numbers), however, Y_test is integer values
# so we need to round() the prediction to integer ones
acc = accuracy_score(prediction.round(), Y_test)
print("Accuracy = {}".format(acc))

In [ ]:
# Confusion matrix
cm = confusion_matrix(Y_test, prediction.round())
sns.heatmap(cm, annot=True)

In [ ]:
print(classification_report(Y_test, prediction.round()))

In [ ]:
#Check results on a few select images
num = np.random.randint(0, X_test.shape[0]-1)
img = X_test[num]
plt.imshow(img)

#Expand features and reshape the input into (num images, x, y, c)
input_img = np.expand_dims(img, axis=0) 
img_feature=feature_extraction(input_img)
img_feature = np.expand_dims(img_feature, axis=0)
input_img_for_lgbm = np.reshape(img_feature, (input_img.shape[0], -1))

#Predict
y_pred = lgbm_model.predict(input_img_for_lgbm)
y_pred = np.argmax(y_pred, axis=0) 
print("The prediction for this image is: ", y_pred)
print("The actual label for this image is: ", Y_test[num])

In [ ]:
#Check results on a few select images
num = np.random.randint(0, X_test.shape[0]-1)
img = X_test[num]
plt.imshow(img)

#Expand features and reshape the input into (num images, x, y, c)
input_img = np.expand_dims(img, axis=0) 
img_feature=feature_extraction(input_img)
img_feature = np.expand_dims(img_feature, axis=0)
input_img_for_lgbm = np.reshape(img_feature, (input_img.shape[0], -1))

#Predict
y_pred = lgbm_model.predict(input_img_for_lgbm)
y_pred = np.argmax(y_pred, axis=0) 
print("The prediction for this image is: ", y_pred)
print("The actual label for this image is: ", Y_test[num])

# Random Forest model

In [ ]:
RF_model = RandomForestClassifier(n_estimators=50, random_state=42)
RF_model.fit(X_for_training, Y_train)
print("Fitting the RF model")

In [ ]:
# Predict the model
y_pred = RF_model.predict(X_for_test)

# Check the accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy = {}".format(accuracy))

In [ ]:
# Feature importance
print(RF_model.feature_importances_)

# Support Vector Machine model

In [ ]:
C = 1.0
SVM_model = svm.SVC(kernel="rbf", C=C).fit(X_for_training, Y_train)
print("Training SVM model")

In [ ]:
# Predict the model
y_pred_svm = SVM_model.predict(X_for_test)

# Check the accuracy
accuracy = accuracy_score(Y_test, y_pred_svm)
print("Accuracy = {}".format(accuracy))

# Comapre the results of trained models

In [ ]:
models = ["LGBM model", "Random Forest model", "SVM model"]

accuracy = [67, 75, 57]
dic = {"Models":models, "Accuracy":accuracy}

df_models = pd.DataFrame.from_dict(dic, orient='index')
df_models.transpose()